In [3]:
import numpy as np
from random import seed
from random import randint

In [4]:
import pandas as pd
import firebase_admin
from firebase_admin import credentials, firestore

In [5]:
import firebase_admin
from firebase_admin import credentials

cred = credentials.Certificate("cv-tailor-2021-firebase-adminsdk-fksao-29fb361d14.json")
firebase_admin.initialize_app(cred)

In [6]:
db = firestore.client()

### THE DICTIONARY WHICH WILL BE RECIEVED

{
name: ,
email: ,
Phone no: ,
linkedin_profile: ,
address: ,
extra_curriculars: list({name: , organization: , description: , skills: list()}),   
education: list({organization: , name_of_course: , qualification: , date_completed: , grade: , skills: list()}),
experience: list({organization: , name_of_role: , date_start: , date_finish: , description: , skills: list()}),
projects: list({name_of_project: , description: , skills: list()}),
}

In [166]:
form_data = { 'name':'Aaryan Gulia', 'email':'aaryan.gulia@gmail.com', 'phone_no':'+974 55994789', 'linkedin_profile':'https://www.linkedin.com/in/aaryan-gulia/', 'address':'Doha, Qatar',
'extra_curriculars': [{'name':'Guitar', 'organization':'At Home', 'description':'I love it', 'skills': ['creativity']}],
'education': [{'organization':'University College London', 'name_of_course':'Theoretical Physics', 'qualification':'MSci', 'date_completed':'2024', 'grade':'1st class', 'skills': ['teamwork skills','critical thinking']}],
'experience': [{'organization':'PPLPWR', 'name_of_role':'Project Lead', 'date_start':'2020', 'date_finish':'', 'description':'I love it', 'skills': ['problem solving','teamwork skills','organization','leaderships skills']}],
'projects': [{'name_of_project':'CVTailor', 'description':'Kidding me!', 'skills': ['computer programming'] }]}

In [167]:
addition = db.collection('persons').document()
addition.set({'name': form_data['name'],'email': form_data['email'],'phone_no':form_data['phone_no'],'linkedin profile':form_data['linkedin_profile'],'address':form_data['address']})

for term in form_data['extra_curriculars']:
    setter = addition.collection('extra_curriculars').document()
    setter.set(term)
    
for term in form_data['education']:
    setter = addition.collection('education').document()
    setter.set(term)

for term in form_data['experience']:
    setter = addition.collection('experience').document()
    setter.set(term)

for term in form_data['projects']:
    setter = addition.collection('projects').document()
    setter.set(term)

In [174]:
get_data = db.collection('persons').where('email','==',form_data['email']).stream()

our_dict = list()
our_list = list()

for data in get_data:
    our_dict = data.to_dict()
    data_id = data.id
    
get_data = db.collection('persons').document(data_id).collection('education').stream()
for data in get_data:
    our_list.append(data.to_dict())
our_dict['education'] = our_list
our_list = list()

get_data = db.collection('persons').document(data_id).collection('extra_curriculars').stream()
for data in get_data:
    our_list.append(data.to_dict())
our_dict['extra_curriculars'] = our_list
our_list = list()

get_data = db.collection('persons').document(data_id).collection('experience').stream()
for data in get_data:
    our_list.append(data.to_dict())
our_dict['experience'] = our_list
our_list = list()

get_data = db.collection('persons').document(data_id).collection('projects').stream()
for data in get_data:
    our_list.append(data.to_dict())
our_dict['projects'] = our_list
our_list = list()

In [175]:
print(our_dict)

{'phone_no': '+974 55994789', 'email': 'aaryan.gulia@gmail.com', 'name': 'Aaryan Gulia', 'linkedin profile': 'https://www.linkedin.com/in/aaryan-gulia/', 'address': 'Doha, Qatar', 'education': [{'grade': '1st class', 'qualification': 'MSci', 'date_completed': '2024', 'skills': ['teamwork skills', 'critical thinking'], 'name_of_course': 'Theoretical Physics', 'organization': 'University College London'}], 'extra_curriculars': [{'organization': 'At Home', 'skills': ['creativity'], 'description': 'I love it', 'name': 'Guitar'}], 'experience': [{'date_finish': '', 'skills': ['problem solving', 'teamwork skills', 'organization', 'leaderships skills'], 'description': 'I love it', 'organization': 'PPLPWR', 'date_start': '2020', 'name_of_role': 'Project Lead'}], 'projects': [{'description': 'Kidding me!', 'skills': ['computer programming'], 'name_of_project': 'CVTailor'}]}


In [172]:
def proper_data(data, profession):
    '''
    input  file
    return python dictionary consisting of json file
    '''
    
    get_skills = db.collection('Careers').where('name','==',profession).stream()
    
    for term in get_skills:
        my_list = []
        get_pro_skills = db.collection('Careers').document(term.id).collection('skills').stream()
        for pro_skills in get_pro_skills:
            my_list.append(pro_skills.to_dict()['name'])
            
    
    new_data = {}
    new_data['name'] = data.get('name')
    new_data['email'] = data.get('email')
    new_data['phone_no'] = data.get('phone_no')
    new_data['extra_curriculars'] = list()
    new_data['experience'] = list()
    new_data['projects'] = list()
    new_data['education'] = list()
    
    for term in new_data['extra_curriculars']:
        for skill in term['skills']:
            if skill in my_list:
                new_data['extra_curriculars'].append(term)
                break
                
    for term in data['experience']:
        for skill in term['skills']:
            if skill in my_list:
                new_data['experience'].append(term)
                break
                
    for term in data['education']:
        for skill in term['skills']:
            if skill in my_list:
                new_data['education'].append(term)
                break
                
    for term in data['projects']:
        for skill in term['skills']:
            if skill in my_list:
                new_data['projects'].append(term)
                break
    
    return new_data

In [176]:
new_dict = proper_data(our_dict,'software engineer')
print(new_dict)

{'name': 'Aaryan Gulia', 'email': 'aaryan.gulia@gmail.com', 'phone_no': '+974 55994789', 'extra_curriculars': [], 'experience': [{'date_finish': '', 'skills': ['problem solving', 'teamwork skills', 'organization', 'leaderships skills'], 'description': 'I love it', 'organization': 'PPLPWR', 'date_start': '2020', 'name_of_role': 'Project Lead'}], 'projects': [{'description': 'Kidding me!', 'skills': ['computer programming'], 'name_of_project': 'CVTailor'}], 'education': [{'grade': '1st class', 'qualification': 'MSci', 'date_completed': '2024', 'skills': ['teamwork skills', 'critical thinking'], 'name_of_course': 'Theoretical Physics', 'organization': 'University College London'}]}
